In [ ]:
import os
import random, string

import numpy as np
np.set_printoptions(precision=3)

import matplotlib.pyplot as plt

#import re

In [ ]:
# pip install soundfile
import soundfile
import librosa
import librosa.display

from IPython.display import display as notebook_display
from IPython.display import Audio as notebook_audio
from IPython.display import HTML as notebook_html

tmp_audio_playback_dir = './data/tmp-audio-playback/'
os.makedirs(tmp_audio_playback_dir, exist_ok=True)

In [ ]:
audio_filenames = [ './librivox/guidetomen_%02d_rowland_64kb.mp3' % (i,) for i in [1,2,3]]
audio_filenames

#mel_filenames = [ f.replace('.mp3', '.melspectra.hkl') for f in audio_filenames ]

In [ ]:
audio_filename = './librivox/guidetomen_02_rowland_64kb.mp3'

audio_samples, _sample_rate = librosa.core.load(audio_filename, sr=None)
audio_samples = audio_samples/np.max(audio_samples)

#audio_samples, 
_sample_rate

In [ ]:
def play_audio_samples(np_samples, autoplay=False):
    hsh = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
    f = tmp_audio_playback_dir + '%s.wav' % (hsh,)
    #def ts(t_mel, end=False):  # t_mel to samples
    #    return int( (t_mel*fft_step + (fft_window if end else 0.))*_sample_rate )
    #samples = audio_samples[ts(s['t_start']):ts(s['t_end'], end=True)]
    soundfile.write(f, np_samples, samplerate=_sample_rate)

    if True or np_samples.shape[0]<100*1000:
        plt.figure(figsize=(12,2))
        librosa.display.waveplot(np_samples, sr=_sample_rate)
        #plt.plot(np_samples)
        #plt.plot(np.arange(s['t_start'],s['t_end'],), audio_span)
        #plt.xticks( np.arange(s['t_start'], s['t_end'], 20.), rotation=90 )
        plt.grid(True)

        plt.show()
        
    notebook_display( 
        #notebook_html('<code id="%s">Javascript Loaded</code>' % (hsh,)),
        notebook_html('<span id="%s"></span>' % (hsh,)),
        notebook_audio(f, rate=_sample_rate, autoplay=autoplay)
    )
    return '#'+hsh

audio_id=play_audio_samples(audio_samples)
audio_id

In [ ]:
# https://www.sitepoint.com/community/t/multiple-clickable-audio-timestamps/273260/9

notebook_html("""<script type="text/Javascript">
function get_next_display_ele(id, selector) {
    var pre_div=$(id);
    var ctrl = pre_div.parent().parent().next().find(selector);
    return $(ctrl)[0];
}
function get_audio_ele(id) {
    return get_next_display_ele(id, 'audio');
}
function get_textbox_ele(id) {
    return get_next_display_ele(id, 'textbox');
}
</script>""")

In [ ]:
txt_filename = audio_filename.replace('.mp3', '.txt')
txt_for_audio = ''
try:
    with open(txt_filename, 'rt') as f:
        txt_for_audio=f.read()
    print("%s loaded" % txt_filename)
except:
    print("%s : File not found" % txt_filename)

In [ ]:
notebook_html("""
<textarea rows=20 cols=70 id={id}>{content}</textarea>
""".format(id=audio_id+'_tb', content=txt_for_audio))

In [ ]:
notebook_html("""<script type="text/Javascript">
var audio_ele=get_audio_ele('{id}');
var textbox_ele=get_textbox_ele('{id}_tb');
//alert( ele.currentTime );  // YES!
</script>""".format(id=audio_id))